In [1]:

from datetime import datetime
import functools
import gymnasium as gym
from agent_template.agent_template import HumanoidTemplate
import mujoco as mj
from utils.SimpleConverter import SimpleConverter
import yaml
from box import Box
import numpy as np

In [2]:
def load_args():
    # Path to your YAML file
    yaml_file_path = 'config_params/punch.yaml'
    # Load the YAML file
    with open(yaml_file_path, 'r') as file:
        args = Box(yaml.safe_load(file))
    
    return args


def load_arguments():
    args = load_args()
    trajectory = SimpleConverter(args.ref)
    trajectory.load_mocap()
    
    return trajectory,args



In [3]:

trajectory,args = load_arguments()


In [21]:


env = HumanoidTemplate(model_path=args.model,reference_data=trajectory,args=args,
                           frame_skip=1, render_mode="human")
observation, info = env.reset()
for _ in range(1000):
    action = env.action_space.sample()  # agent policy that uses the observation and info     
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        #observation, info = env.reset()
        print('end')
        break

env.close()
    

this is the model_path assets/models/final_humanoid.xml
Reward weights - w_p: 0.65, w_v: 0.1, w_e: 0.15, w_c: 0.1
Deep Mimic weights - w_pose: 2, w_angular: 0.1, w_efector: 40, w_com: 10
end


In [5]:
print(env.model.opt.timestep)

print(env.rollout_lenght)
print(env.cycle_len)

0.02
65
130


In [6]:
print(env.data.subtree_com[0,:])
print(env.reference_data.subtree_com[0,:])

[0.03125177 0.06669548 0.23453194]
[1.20056004 1.2696906  0.83045276]


In [7]:
env.action_space

Box(-0.5, 0.5, (28,), float32)

In [8]:
print(env.data.geom_xpos[env.dict_ee])
print(env.reference_data.geom_xpos[env.dict_ee])

[[-0.00672478  0.06839397  0.04486527]
 [-0.08886388 -0.02545323  0.46860531]
 [-0.78951892  0.34453823  0.17031632]
 [-0.79358615  0.3136229   0.37956955]]
[[1.53434276 1.13005327 1.12301081]
 [1.14193208 1.41035345 0.94175013]
 [1.20941731 1.20669663 0.04990311]
 [0.95184073 1.25340477 0.12560501]]


In [9]:
from some_math import rfctransformations
rfctransformations.quaternion_from_euler



<function some_math.rfctransformations.quaternion_from_euler(ai, aj, ak, axes='sxyz')>

In [10]:
body_names = [mj.mj_id2name(env.model, mj.mjtObj.mjOBJ_BODY, i) for i in range(env.model.nbody)]

print(len(body_names[1:]))


13


In [11]:
env.model.nbody

14

In [12]:
body_qposaddr = {
            mj.mj_id2name(env.model, mj.mjtObj.mjOBJ_BODY, i): (env.model.body_jntadr[i], env.model.body_jntadr[i] + env.model.body_jntnum[i])
            for i in range(env.model.nbody)
        }

In [13]:
body_qposaddr

{'world': (-1, -1),
 'root': (0, 1),
 'chest': (1, 4),
 'neck': (4, 7),
 'right_shoulder': (7, 10),
 'right_elbow': (10, 11),
 'left_shoulder': (11, 14),
 'left_elbow': (14, 15),
 'right_hip': (15, 18),
 'right_knee': (18, 19),
 'right_ankle': (19, 22),
 'left_hip': (22, 25),
 'left_knee': (25, 26),
 'left_ankle': (26, 29)}

In [14]:
def get_body_quat(data):
    qpos = data.qpos.copy()
    body_quat = [qpos[3:7]]
    for body in body_names[1:]:
        print(body)
        if body == 'root' or not body in body_qposaddr:
            continue
        start, end = body_qposaddr[body]
        euler = np.zeros(3)
        euler[:end - start] = qpos[start:end]
        quat = rfctransformations.quaternion_from_euler(euler[0], euler[1], euler[2])
        body_quat.append(quat)
    body_quat = np.concatenate(body_quat)
    return body_quat



In [15]:
env.reference_data.qpos

array([ 1.15749186,  1.2052196 ,  0.779908  ,  0.93956125, -0.04013979,
        0.0455905 ,  0.33694951, -0.0274556 ,  0.33943512, -0.07411261,
        0.24039915, -0.14672062, -0.65170114, -0.44824064, -0.04377439,
       -0.3751392 ,  2.18266927,  0.66939759,  0.21359398, -0.8169909 ,
        2.01410563,  0.07981827, -0.67394493, -0.10625215, -1.12973068,
       -0.1572688 , -0.26515211, -0.19503912,  0.29883702, -0.61212537,
        0.19371584, -1.46266445, -0.03983083,  0.16201492, -0.032911  ])

In [16]:
quat = get_body_quat(env.data)
quat


root
chest
neck
right_shoulder
right_elbow
left_shoulder
left_elbow
right_hip
right_knee
right_ankle
left_hip
left_knee
left_ankle


array([-0.5533727 , -0.58166088, -0.43361794, -0.40917571,  0.95670869,
        0.04143719,  0.09133048, -0.27322187,  0.90353453, -0.31606276,
       -0.14489016, -0.25047261,  0.99472867, -0.09967544, -0.0225187 ,
       -0.00852029,  0.98858638, -0.15065515,  0.        ,  0.        ,
        0.99396608, -0.07005027, -0.06248889,  0.05674088,  0.99936839,
        0.03553614,  0.        ,  0.        ,  0.98202579,  0.15483359,
        0.030558  ,  0.10352838,  0.99189024,  0.12709738,  0.        ,
        0.        ,  0.99585179,  0.08999473,  0.00537175,  0.0123002 ,
        0.99929474,  0.02835496, -0.01490668,  0.0195909 ,  0.99940491,
        0.03449397,  0.        ,  0.        ,  0.99867465, -0.01617723,
        0.0078937 ,  0.04821748])

In [17]:
n_joint = quat.shape[0]//4
n_joint

13

In [18]:
env.data.xpos[1:]

array([[ 0.02669728,  0.08389232,  0.20952685],
       [ 0.25404032,  0.02080729,  0.19939195],
       [ 0.46153429, -0.05409555,  0.23765863],
       [ 0.50351723, -0.08725964,  0.05981138],
       [ 0.2396838 , -0.01100098,  0.05059266],
       [ 0.43987093, -0.07936626,  0.42037196],
       [ 0.16711838, -0.05589543,  0.44411236],
       [ 0.02389593,  0.08741524,  0.12475926],
       [-0.38368176,  0.19291533,  0.14600054],
       [-0.78002931,  0.29526604,  0.16664912],
       [ 0.02949863,  0.08036941,  0.29429444],
       [-0.37535897,  0.18954536,  0.3375606 ],
       [-0.77573852,  0.26658357,  0.37945035]])

In [19]:
initial = env.reference_trajectory_qpos[0]
from some_math import rfcmath

prev = get_body_quat(env.data)
current= get_body_quat(env.data)
print(current.shape)
ang = rfcmath.get_angvel_fd(prev,current,env.dt)

print(ang.shape)


root
chest
neck
right_shoulder
right_elbow
left_shoulder
left_elbow
right_hip
right_knee
right_ankle
left_hip
left_knee
left_ankle
root
chest
neck
right_shoulder
right_elbow
left_shoulder
left_elbow
right_hip
right_knee
right_ankle
left_hip
left_knee
left_ankle
(52,)
(39,)


In [20]:
from some_math import rfcmath

qvel = env.data.qvel
qpos = env.data.qpos

qvel[:3] = rfcmath.transform_vec(qvel[:3],qpos[3:7], 'root').ravel()

obs = []

obs.append(qpos[2:])

obs.append(qvel)
#plus phi on the env
phi = (5% 130)/30

obs.append([phi])

obs = np.concatenate(obs)
obs.shape
#size 68




(68,)